In [ ]:
from nbtemplate import display_header
display_header('ExplainBending.ipynb')

# Why bending a grating is useful

In [ ]:
import pathlib
import numpy as np
import matplotlib.pyplot as plt
from marxs.optics import CATGrating, FlatDetector, OrderSelector
from marxs.utils import generate_test_photons

%matplotlib inline

In [ ]:
import sys
sys.path.append('../')
from redsox.gratings import bend_gratings

In [ ]:
cat = CATGrating(zoom=[1, 500, 500], position=[1000, 0,0 ], d=1e-4, order_selector=OrderSelector([1]))
cat2 = CATGrating(zoom=[1, 500, 500], position=[1000, 0,0 ], d=1e-4, order_selector=OrderSelector([1]))

det = FlatDetector(zoom=[1, 50, 50])

In [ ]:
from transforms3d.axangles import axangle2mat

def blazemat(blazeang):
    return axangle2mat(np.array([0, 0, 1]), blazeang)

catb = CATGrating(zoom=[1, 500, 500], position=[1000, 0,0 ], orientation=blazemat(-.2),
                  d=1e-4, order_selector=OrderSelector([1]))
catb2 = CATGrating(zoom=[1, 500, 500], position=[1000, 0,0 ], orientation=blazemat(-.2),
                  d=1e-4, order_selector=OrderSelector([1]))

In [ ]:
phot_in = generate_test_photons(10)
angs = np.linspace(-.5, .1, 10)
r0 = 1200
phot_in['pos'] = np.stack([r0 * np.cos(angs), r0 * np.sin(angs), np.zeros_like(angs), np.ones_like(angs)]).T
phot_in['dir'] = -phot_in['pos']
phot_in['dir'][:, -1] = 0
phot_in['dir'] = phot_in['dir'] / np.linalg.norm(phot_in['dir'], axis=1)[:, None]

In [ ]:
cat2.elements = []  # Because the bending function expects sub-elements which we don't need here
bend_gratings([cat2], 1000)

In [ ]:
catb2.elements = []  # Because the bending function expects sub-elements which we don't need here
bend_gratings([catb2], 1000)

In [ ]:
fig, axes = plt.subplots(subplot_kw={'aspect':'equal'}, ncols=2)
for ax in axes:
    rays = ax.plot(np.stack([phot_in['pos'][:, 1], np.zeros(10)]),
               np.stack([phot_in['pos'][:, 0], np.zeros(10)]), 
               'k')
out = np.stack([cat.geometry['center'] + cat.geometry['v_y'], cat.geometry['center'] - cat.geometry['v_y']])
axes[0].plot(out[:, 1], out[:, 0], lw=3)
out = cat2.geometry.parametric_surface(z=[0])
axes[0].plot(out[:, 0, 1], out[:, 0, 0], lw=3)

out = np.stack([catb.geometry['center'] + catb.geometry['v_y'], catb.geometry['center'] - catb.geometry['v_y']])
axes[1].plot(out[:, 1], out[:, 0], lw=3)
out = catb2.geometry.parametric_surface(z=[0])
axes[1].plot(out[:, 0, 1], out[:, 0, 0], lw=3)
axes[0].set_title('no blaze')
out = axes[1].set_title('blaze angle')

for ax in axes:
    ax.set(yticklabels=[], xticklabels=[])  # remove the tick labels
    ax.tick_params(left=False, bottom=False)  # remove the ticks

pathlib.Path('../figures/explainbending.pdf').mkdir(parents=True, exist_ok=True)
fig.savefig('../figures/explainbending.pdf', bbox_inches='tight')

In a converging beam (black rays) the angle between the grating normal and the rays changes with grating position for a flat grating (blue). Bending the grating can compensate for this problem (orange) for gratings that are small compared to the radius. This is true for gratings where the ray through the grating center is normal to the grating (left) and for gratings with a design blaze angle (right).

In a CAT (critical angle transmission) grating, the diffraction efficiency (the probability that a diffracted photon ends up in a specific order) depends on the blaze angle (the angle between the grating normal and the incoming photon). For CAT gratings, the most likely diffraction angle is about twice the blaze angle. In our design, only the photons in first order are matched to the peak reflectivity of the multi-layer mirror, while photons at higher orders will not hit a detector. Thus, it is imperative to optimize the number photons diffracted into first order. To achive this, the gratings are blazed by design, such that photons have a blaze angle about 0.8 degrees. Holding this blaze angle over the entire surface of a grating in a converging photon beam requires the grating to be bend. To first order, the grating can be rolled up on a cylindrcal surface (right). In practice, radii and blaze angles are much smaller than shown here, so that this approximation is valid. The figure on the right has an extremely large grating compared to the distance from the focal point and we can thus see that in this case, the blaze angle changes from left to right, although to a lesser degree than for the flat grating.

In [ ]:
fig, ax = plt.subplots(subplot_kw={'aspect':'equal'})
rays = ax.plot(np.stack([phot_in['pos'][:, 1], np.zeros(10)]),
               np.stack([phot_in['pos'][:, 0], np.zeros(10)]), 
               'k')
out = np.stack([cat.geometry['center'] + cat.geometry['v_y'], cat.geometry['center'] - cat.geometry['v_y']])
ax.plot(out[:, 1], out[:, 0], lw=3)
out = cat2.geometry.parametric_surface(z=[0])
ax.plot(out[:, 0, 1], out[:, 0, 0], lw=3)

ax.set(yticklabels=[], xticklabels=[])  # remove the tick labels
ax.tick_params(left=False, bottom=False)  # remove the ticks


fig.savefig('../figures/explainbending_1.pdf', bbox_inches='tight')